# Question 4: Binary, Hex, and Decimal

## 4(a) Explain these functions

In [2]:
using Random
Random.seed!(1)

random_binary_string(n = 12) = *([rand(['0','1']) for _ in 1:n]...)
random_hex_string(n=3) = *([rand(union('0':'9','A':'F')) for _ in 1:n]...);

@show random_binary_string()
@show random_hex_string();

random_binary_string() = "001110111010"
random_hex_string() = "409"


### `random_binary_string`

The input of this function is a string length, `n`; and the output is a string of random 1s and 0s, of length `n`. `rand(['0', '1'])` randomly selects and returns the char '0' or '1'. An array comprehension is used to generate `n` (12, by default) of these characters, and they get stored together in a `Vector{Char}`. The splat operator, `...`, then unpacks this array into `n` individual `Char` arguments, which are then sent to the string concatonation operator, `*`, joining them into one string.

### `random_hex_string`

The input of this function is a string length, `n`; and the output is a string containing a random hexadecimal (base-16) number. The first (maybe, innermost?) part of the function `union('0':'9','A':'F')`, creates a 16 element `Vector{Char}`. The objects '0':'9' and 'A':'F' are ranges, which, when passed to the union function, are unpacked into arrays, and the union of the two arrays is taken. `rand(union('0':'9','A':'F'))` randomly selects and returns one of these characters. An array comprehension is then used to select `n` (3, by default) of these random characters and put them in their very own `Vector{Char}`. Finally, the same splat+concatonation trick is used to convert the vector of characters into a single string, which effectively contains a hexadecimal number.

## 4(b) Parse these arrays

In [33]:
using Random
Random.seed!(1)
r_bin_strs = [random_binary_string() for _ in 1:10^3]
r_hex_strs = [random_hex_string() for _ in 1:10^3];

println("Sum of Binary Nums:\t$(sum([parse(Int, num, base=2) for num in r_bin_strs]))")
println("Sum of HexDec Nums:\t$(sum([parse(Int, num, base=16) for num in r_hex_strs]))")

Sum of Binary Nums:	2035713
Sum of HexDec Nums:	2111429


Guessing these will be fun! Ok.

The binary number has 12 digits, which means the maximum number will be $2^{12} - 1 = 4095$. Since the randomiser is $iid$ uniformly, I guess that the average number will be half, so 2047.5. Multiplying this by 1000, i guess that the sum will be aroud 2,047,500

I use the same methodology and assumptions to estimate the hex number. The max is $16^{3} - 1 = 4095$, so the sum of adding up 1000 of these, I estimate, will also be 2,047,500

In the end, I think I was pretty close!
$$
    \text{Binary: } \frac{2,047,500 - 2,035,713}{\left\lvert 2,035,713 \right\rvert} \approx 0.5791\% \qquad \text{HexDec: } \frac{2,047,500 - 2,111,429}{\left\lvert 2,111,429 \right\rvert} \approx -3.0277\% 
$$
My estimate was half a percentage point under the actual sum for the binary numbers, and 3% under the sum of the hexadecimal numbers.

## 4(c) Create your own converter

In [94]:
function mks_binToDec(str::String)
    # Original Code
    # res::Int32 = 0
    # for (p, c) in zip(length(str)-1:-1:0, str)
    #     res += c == '1' ? 2^p : 0
    # end
    # return res
    #
    # BUT I LOVE A GOOD/DUMB/CRYPTIC LIST COMPREHENSION
    return sum([c == '1' ? 2^p : 0 for (p, c) in zip(length(str)-1:-1:0, str)])
end

# Original Code
# function mks_hexdecToDec(str::String)
    # res::Int32 = 0
    # hexLookup = Dict([('0', 0), ('1', 1), ('2', 2), ('3', 3), ('4', 4), ('5', 5), ('6', 6), ('7', 7), ('8', 8), ('9', 9), ('A', 10), ('B', 11), ('C', 12), ('D', 13), ('E', 14), ('F', 15)])
    # for (p, c) in zip(length(str)-1:-1:0, str)
    #     res += get(hexLookup, c, 0) * 16^p
    # end
    # return res
# end
# Seriously, I'm a feind. We use a let block so that we don't reinitalise a hashtable everytime we run the function.
let hexLookup = Dict([('0', 0), ('1', 1), ('2', 2), ('3', 3), ('4', 4), ('5', 5), ('6', 6), ('7', 7), ('8', 8), ('9', 9), ('A', 10), ('B', 11), ('C', 12), ('D', 13), ('E', 14), ('F', 15)])
    global mks_hexdecToDec
    function mks_hexdecToDec(str::String)
        return sum([get(hexLookup, c, 0) * 16^p for (p, c) in zip(length(str)-1:-1:0, str)])
    end
end

using Random
Random.seed!(1)
r_bin_strs = [random_binary_string() for _ in 1:10^3]
r_hex_strs = [random_hex_string() for _ in 1:10^3];

println("Sum of Binary Nums (og parse):\t$(sum([parse(Int, num, base=2) for num in r_bin_strs]))")
println("Sum of Binary Nums (my versi):\t$(sum([mks_binToDec(num) for num in r_bin_strs]))\n")
println("Sum of HexDec Nums (og parse):\t$(sum([parse(Int, num, base=16) for num in r_hex_strs]))")
println("Sum of HexDec Nums (my versi):\t$(sum([mks_hexdecToDec(num) for num in r_hex_strs]))")

Sum of Binary Nums (og parse):	2035713
Sum of Binary Nums (my versi):	2035713

Sum of HexDec Nums (og parse):	2111429
Sum of HexDec Nums (my versi):	2111429


This was a fun task. I realised that `1:length(str)` will increment the exponent from left to right, when numbers are read from right to left. My inital solution involved me reversing the string, but I decided it would be better to just step from the largest number down to 0, backwards. The binary converter is clean, but I really wanted to avoid having a tower of `if... elseif ... else` in my hexdec converter. A wise man once said "when in doubt, use a hashtable," so thats what I did. Though, it's possible that, as is the case in my original code, I don't think that initialising a dictonary everytime you call the function is more performant than the `if` tower. C++ has a wonderful feature, where you can initalise a function-scope variable with the `static` keyword, and it remains mutable in memory for the lifetime of the program, and only the function can see it and modify it. So good for these purposes. I looked around to try and find something like this in Julia, and the best I could come up with is this `let` keyword, which initalises the dictonary once, and lets me define a function in global namespace which has access to that variable. Kind of close, its a little clunkier then C++'s `static`. But rambling aside, the functions are implemented, and there is 0 delta between my sum and the original `parse`'s sums!